In [ ]:
import numpy as np
import os
from PIL import Image # Usaremos PIL solo para cargar y procesar la imagen. Es el mínimo indispensable.

# --- Funciones de Activación ---
# Las funciones de activación introducen no-linealidad, permitiendo a la red aprender patrones complejos.

def relu(x):
    """Función de activación ReLU."""
    return np.maximum(0, x)

def relu_derivative(x):
    """Derivada de ReLU para la retropropagación."""
    return np.where(x > 0, 1, 0)

def sigmoid(x):
    """Función de activación Sigmoid (útil para la capa de salida del autoencoder)."""
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """Derivada de Sigmoid."""
    s = sigmoid(x)
    return s * (1 - s)

# --- Clase para una Capa Densa ---
# Esta es la unidad de construcción fundamental de nuestra red.
class DenseLayer:
    def __init__(self, input_size, output_size, activation='relu'):
        # Inicializamos los pesos con valores pequeños y aleatorios y los sesgos en cero.
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.zeros((1, output_size))
        self.activation_name = activation
        
        # Variables para guardar valores durante el forward pass para usarlos en el backward pass
        self.input = None
        self.z = None # Salida lineal (antes de la activación)

    def forward(self, input_data):
        """Paso hacia adelante: calcula la salida de la capa."""
        self.input = input_data
        self.z = np.dot(self.input, self.weights) + self.biases
        
        if self.activation_name == 'relu':
            return relu(self.z)
        elif self.activation_name == 'sigmoid':
            return sigmoid(self.z)
        return self.z # Sin activación para la capa de salida del clasificador

    def backward(self, output_error, learning_rate):
        """Paso hacia atrás: calcula los gradientes y actualiza los pesos."""
        if self.activation_name == 'relu':
            delta = output_error * relu_derivative(self.z)
        elif self.activation_name == 'sigmoid':
            delta = output_error * sigmoid_derivative(self.z)
        else:
            delta = output_error

        # Gradiente de los pesos y sesgos
        weights_gradient = np.dot(self.input.T, delta)
        biases_gradient = np.sum(delta, axis=0, keepdims=True)
        
        # Gradiente de entrada para la capa anterior
        input_error = np.dot(delta, self.weights.T)
        
        # Actualización de pesos y sesgos (aquí es donde ocurre el "aprendizaje")
        self.weights -= learning_rate * weights_gradient
        self.biases -= learning_rate * biases_gradient
        
        return input_error

# --- Función para cargar y pre-procesar datos ---
# Aplanamos la imagen en un vector y la normalizamos.
def load_and_preprocess_image(path, size=(64, 64)):
    img = Image.open(path).convert('L') # Convertir a escala de grises para simplificar
    img = img.resize(size)
    img_array = np.array(img, dtype=float)
    img_array /= 255.0  # Normalizar píxeles a un rango de 0 a 1
    return img_array.flatten().reshape(1, -1) # Aplanar y convertir en vector fila

In [ ]:
# Autoencoder
class Autoencoder:
    def __init__(self, input_size, encoding_size):
        self.encoder = DenseLayer(input_size, encoding_size, activation='relu')
        self.decoder = DenseLayer(encoding_size, input_size, activation='sigmoid') # Sigmoid para salida entre 0 y 1

    def forward(self, x):
        encoded = self.encoder.forward(x)
        decoded = self.decoder.forward(encoded)
        return decoded

    def backward(self, output_error, learning_rate):
        error = self.decoder.backward(output_error, learning_rate)
        self.encoder.backward(error, learning_rate)

    def train(self, data, epochs, learning_rate):
        # El Autoencoder se entrena con imágenes 'normales'
        print("Entrenando Autoencoder...")
        for epoch in range(epochs):
            total_loss = 0
            for x in data: # x es una imagen aplanada
                # Forward pass
                reconstructed = self.forward(x)
                
                # Calcular el error (Mean Squared Error)
                loss = np.mean((x - reconstructed)**2)
                total_loss += loss
                
                # Backward pass
                output_error = 2 * (reconstructed - x) / x.size
                self.backward(output_error, learning_rate)
            
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(data)}")

# --- Preparación y Entrenamiento del Autoencoder ---

NORMAL_IMG_DIR = 'dataset/train/normal/'
normal_images = [load_and_preprocess_image(os.path.join(NORMAL_IMG_DIR, f)) for f in os.listdir(NORMAL_IMG_DIR)]

IMG_SIZE = 64 * 64
ENCODING_SIZE = 32 # Tamaño de la representación comprimida (un hiperparámetro a ajustar)

# Crear y entrenar el autoencoder
autoencoder = Autoencoder(input_size=IMG_SIZE, encoding_size=ENCODING_SIZE)
# autoencoder.train(normal_images, epochs=50, learning_rate=0.01) # Descomentar para entrenar

In [ ]:
#Clasificador de Anomalías
class AnomalyClassifier:
    def __init__(self, input_size, hidden_size, num_classes):
        self.layer1 = DenseLayer(input_size, hidden_size, activation='relu')
        self.layer2 = DenseLayer(hidden_size, num_classes, activation='none') # Salida lineal (logits)

    def forward(self, x):
        x = self.layer1.forward(x)
        x = self.layer2.forward(x)
        return x

    def backward(self, error, learning_rate):
        error = self.layer2.backward(error, learning_rate)
        self.layer1.backward(error, learning_rate)

    def train(self, data, labels, epochs, learning_rate):
        print("Entrenando Clasificador de Anomalías...")
        num_classes = len(np.unique(labels))
        for epoch in range(epochs):
            total_loss = 0
            for x, y_true_idx in zip(data, labels):
                # Forward pass
                logits = self.forward(x)
                
                # Softmax y Cross-Entropy Loss
                exp_scores = np.exp(logits - np.max(logits))
                probs = exp_scores / np.sum(exp_scores)
                
                loss = -np.log(probs[0, y_true_idx])
                total_loss += loss
                
                # Backward pass (gradiente de cross-entropy con softmax)
                probs[0, y_true_idx] -= 1
                error = probs / len(data)
                
                self.backward(error, learning_rate)
            
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(data)}")


# --- Preparación y Entrenamiento del Clasificador ---

ANOMALY_DIRS = {'incendios': 0, 'basura': 1, 'zonas_secas': 2}
anomaly_images = []
anomaly_labels = []

for label_name, label_idx in ANOMALY_DIRS.items():
    dir_path = f'dataset/train/{label_name}/'
    for f in os.listdir(dir_path):
        anomaly_images.append(load_and_preprocess_image(os.path.join(dir_path, f)))
        anomaly_labels.append(label_idx)

# Crear y entrenar el clasificador
classifier = AnomalyClassifier(input_size=IMG_SIZE, hidden_size=128, num_classes=len(ANOMALY_DIRS))
# classifier.train(anomaly_images, anomaly_labels, epochs=100, learning_rate=0.01) # Descomentar para entrenar

In [ ]:
#Red entrenada
def analyze_forest_image(image_path, autoencoder_model, classifier_model, anomaly_threshold=0.015):
    """
    Analiza una imagen usando el sistema de dos etapas.
    """
    # 1. Cargar y pre-procesar la imagen
    img_vector = load_and_preprocess_image(image_path)
    
    # --- Etapa 1: Detector de Anomalías ---
    reconstructed_img = autoencoder_model.forward(img_vector)
    reconstruction_error = np.mean((img_vector - reconstructed_img)**2)
    
    print(f"Error de reconstrucción: {reconstruction_error:.5f}")
    
    if reconstruction_error < anomaly_threshold:
        return "Normal", f"El bosque parece saludable (Error: {reconstruction_error:.5f})"
    else:
        # --- Etapa 2: Clasificador de Problemas ---
        logits = classifier_model.forward(img_vector)
        
        # Obtener la predicción final
        predicted_class_idx = np.argmax(logits)
        class_names = list(ANOMALY_DIRS.keys())
        predicted_class_name = class_names[predicted_class_idx]
        
        return "Anomalía Detectada", f"Posible problema: {predicted_class_name.capitalize()}"

In [ ]:
#Backend (todavía no lo tenemos xd)